# **Retail Sales Analysis Notebook**

## Objectives

* Write your notebook objective here, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write down which data or information you need to run the notebook 

* Here i will explain df_raw

## Tranformations

* Here I will explain df_transform

## Outputs

* Here I will explain denormalised_df (clean) and promo_denormalised_df (clean)

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Data Extract & Dictionary Creation

In [ ]:
# Core Libraries
import pandas as pd                        # Data manipulation and analysis
import numpy as np                         # Numerical computing and operations

# Visualization Libraries
import matplotlib.pyplot as plt            # Basic plotting and charting
import seaborn as sns                      # Statistical data visualisation

In [2]:
# Directory Management
import os                                  # Import os for data directory management
import shutil                              # Import shutil for file operations

# Data Acquisition Library
import kagglehub                           # Import Kaggle Hub to Download retail datasets

# Download dataset 
path = kagglehub.dataset_download("manjeetsingh/retaildataset")
print(f'Files downloaded to: {path}')

# Copy CSV files to data directory
os.makedirs("../data", exist_ok=True)
for file in os.listdir(path):
    if file.endswith(".csv"):
        shutil.copy2(os.path.join(path, file), f"../data/{file.lower().replace(' ', '-')}")

# backlog feat: remove files from the kaggle cashe folder on copy see ≈
# https://github.com/users/Julian-Elliott/projects/3/views/1?pane=issue&itemId=115149029&issue=Julian-Elliott%7Cretail-sales-analysis%7C13
print("Files copied to ../data")

/Users/julianelliott/Documents/GitHub/retail-sales-analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files downloaded to: /Users/julianelliott/.cache/kagglehub/datasets/manjeetsingh/retaildataset/versions/2
Files copied to ../data


In [3]:
# Load the retail datasets into dataframes
sales_df_raw = pd.read_csv("../data/sales-data-set.csv")
stores_df_raw = pd.read_csv("../data/stores-data-set.csv") 
features_df_raw = pd.read_csv("../data/features-data-set.csv")

### Inspecting Dataset Data and building a data dictionary

In [4]:
# Custom Function to create a comprehensive data dictionary for multiple datasets as I experienced issues with ydata-profiling
def create_data_dictionary(datasets_dict):
    # Quick ref for detailed descriptions for each known column (from Kaggle data card https://www.kaggle.com/datasets/manjeetsingh/retaildataset/data)
    descriptions = {
        'Store': 'The store number',
        'Date': 'The week start date',
        'Temperature': 'Average temperature in the region',
        'Fuel_Price': 'Cost of fuel in the region',
        'MarkDown1': 'Anonymized promotional markdown data (after Nov 2011)',
        'MarkDown2': 'Anonymized promotional markdown data (after Nov 2011)',
        'MarkDown3': 'Anonymized promotional markdown data (after Nov 2011)',
        'MarkDown4': 'Anonymized promotional markdown data (after Nov 2011)',
        'MarkDown5': 'Anonymized promotional markdown data (after Nov 2011)',
        'CPI': 'The consumer price index',
        'Unemployment': 'The unemployment rate',
        'IsHoliday': 'Whether the week is a special holiday week',
        'Dept': 'The department number',
        'Weekly_Sales': 'Sales for the given department in the given store',
        'Type': 'Store type classification',
        'Size': 'Store size in square feet'
    }
    
    dictionary_data = []
    for dataset_name, df in datasets_dict.items():
        for column in df.columns:
            # Get 3 sample values (non-null)
            sample_values = df[column].dropna().head(3).tolist()
            sample_str = ', '.join([str(x) for x in sample_values])
            
            dictionary_data.append({
                'Dataset': dataset_name,
                'Column': column,
                'Data Type': str(df[column].dtype),
                'Row Count': len(df),
                'Missing Values': df[column].isnull().sum(),
                'Missing %': round((df[column].isnull().sum() / len(df)) * 100, 2),
                'Unique Values': df[column].nunique(),
                'Sample Values': sample_str,
                'Description': descriptions.get(column, 'Description needed')
            })
    return pd.DataFrame(dictionary_data)

# Create datasets dictionary
datasets = {
    'Sales': sales_df_raw,
    'Stores': stores_df_raw,
    'Features': features_df_raw
}

# Generate data dictionary that can be called upon later
data_dictionary = create_data_dictionary(datasets)

print('\nRaw data Dictionary (before transformation)')
# Display data dictionary
data_dictionary


Raw data Dictionary (before transformation)


,Dataset,Column,Data Type,Row Count,Missing Values,Missing %,Unique Values,Sample Values,Description
0,Sales,Store,int64,421570,0,0.00,45,"1, 1, 1",The store number
1,Sales,Dept,int64,421570,0,0.00,81,"1, 1, 1",The department number
2,Sales,Date,object,421570,0,0.00,143,"05/02/2010, 12/02/2010, 19/02/2010",The week start date
3,Sales,Weekly_Sales,float64,421570,0,0.00,359464,"24924.5, 46039.49, 41595.55",Sales for the given department in the given store
4,Sales,IsHoliday,bool,421570,0,0.00,2,"False, True, False",Whether the week is a special holiday week
5,Stores,Store,int64,45,0,0.00,45,"1, 2, 3",The store number
6,Stores,Type,object,45,0,0.00,3,"A, A, B",Store type classification
7,Stores,Size,int64,45,0,0.00,40,"151315, 202307, 37392",Store size in square feet
8,Features,Store,int64,8190,0,0.00,45,"1, 1, 1",The store number
9,Features,Date,object,8190,0,0.00,182,"05/02/2010, 12/02/2010, 19/02/2010",The week start date


#### Potential Issues shown in the Data Dictionary

Markdown columns `1` through `5`, `CPI` and `Unemployment` in the `Features` dataset presents notable data quality challenges:

- **Missing Data:**  
  Each MarkDown column contains a significant percentage of missing values (ranging from 50% to over 64%). Part of this is expected, as promotional markdown data is only available after November 2011, and earlier records do not include these values.  
  To a lesser extent, the `CPI` (Consumer Price Index) and `Unemployment` columns are also missing about 7% of their values. While this is less dramatic, it may still impact analyses involving economic features of our dataset.
  
- **Time-Dependent Availability:**  
  The fact that MarkDown data is only present for dates after November 2011 (as explained in the kaggle [data card](https://www.kaggle.com/datasets/manjeetsingh/retaildataset/data)) means that analyses involving these columns must account for their partial availability. Any models or insights involving markdowns will be biased toward the later part of the dataset and may not generalise to earlier periods.

- **Other Features:**  
  Other columns such as `Store`, `Date`, `Temperature`, `IsHoliday` and `Weekly_Sales` have no missing values and can be used with greater confidence in a general analyses.

In [5]:
# Create temporary dataframes to avoid modifying the original data before transform stage
sales_df_transform = sales_df_raw.copy()
features_df_transform = features_df_raw.copy()
stores_df_transform = stores_df_raw.copy()

# Convert temp date columns with correct format (DD/MM/YYYY)
sales_df_transform['Date'] = pd.to_datetime(sales_df_transform['Date'], format='%d/%m/%Y')
features_df_transform['Date'] = pd.to_datetime(features_df_transform['Date'], format='%d/%m/%Y')

# Find the first date where any MarkDown data is available
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
first_markdown_date = features_df_transform[features_df_transform[markdown_cols].notna().any(axis=1)]['Date'].min()

In [6]:
# Create a summary dataframe of the datasets with date ranges and markdown info
dataset_summary = pd.DataFrame({
    'Dataset': ['sales-data-set.csv', 'features-data-set.csv', 'markdown-features'],
    'Records': [
        sales_df_transform.shape[0], 
        features_df_transform.shape[0], 
        features_df_transform[features_df_transform[markdown_cols].notna().any(axis=1)].shape[0]
    ],
    'Start_Date': [
        sales_df_transform["Date"].min().strftime("%Y-%m-%d"),  # Sales data start date using transform df
        features_df_transform["Date"].min().strftime("%Y-%m-%d"),  # Features data start date using transform df
        first_markdown_date.strftime("%Y-%m-%d")  # First date with markdown data
    ],
    'End_Date': [
        sales_df_transform["Date"].max().strftime("%Y-%m-%d"),  # Sales data end date using transform df
        features_df_transform["Date"].max().strftime("%Y-%m-%d"),  # Features data end date using transform df
        features_df_transform["Date"].max().strftime("%Y-%m-%d")  # Markdown data shares same end date as features
    ],
    'Description': [
        f'Weekly sales data spanning {stores_df_raw.shape[0]} stores',  # Sales dataset description
        'Economic and promotional features',  # Features dataset description
        f'Promotional markdown data'  # Markdown subset description
    ]
})

print('\nComparison of date ranges across datasets:')

dataset_summary


Comparison of date ranges across datasets:


,Dataset,Records,Start_Date,End_Date,Description
0,sales-data-set.csv,421570,2010-02-05,2012-10-26,Weekly sales data spanning 45 stores
1,features-data-set.csv,8190,2010-02-05,2013-07-26,Economic and promotional features
2,markdown-features,4050,2011-11-11,2013-07-26,Promotional markdown data


In [7]:
# Use data dictionary to identify matching columns between transform datasets
transform_datasets = {
    'Sales_Transform': sales_df_transform,
    'Stores_Transform': stores_df_transform, 
    'Features_Transform': features_df_transform
}

# Generate data dictionary for transform datasets
transform_dictionary = create_data_dictionary(transform_datasets)

# Find columns that appear in multiple datasets
column_counts = transform_dictionary['Column'].value_counts()
shared_columns = column_counts[column_counts > 1]

print("SHARED COLUMNS (JOIN KEYS):")
# Only show rows for columns that appear in multiple datasets
shared_dict = transform_dictionary[transform_dictionary['Column'].isin(shared_columns.index)]
shared_dict[['Dataset', 'Column', 'Data Type', 'Unique Values']].sort_values('Column')


SHARED COLUMNS (JOIN KEYS):


,Dataset,Column,Data Type,Unique Values
2,Sales_Transform,Date,datetime64[ns],143
9,Features_Transform,Date,datetime64[ns],182
4,Sales_Transform,IsHoliday,bool,2
19,Features_Transform,IsHoliday,bool,2
0,Sales_Transform,Store,int64,45
5,Stores_Transform,Store,int64,45
8,Features_Transform,Store,int64,45


### Data Relationships Between Tables

The retail dataset appears to follow a **Star Schema** pattern with clear relationships between the three main tables:

#### **Join Keys Identified**
- **Sales ↔ Stores**: `Store` (Many-to-One relationship)
- **Sales ↔ Features**: `Store` + `Date` composite key (One-to-One relationship)

#### **Table Roles**
- **SALES**: Central fact table containing transactional data
  - Granularity: Store + Department + Week level
  - Contains measurable metrics (Weekly_Sales)
  
- **STORES**: Dimension table with store characteristics
  - Granularity: Store level (static attributes)
  - Contains: Store type, size, and other store properties
  
- **FEATURES**: Dimension table with temporal/environmental factors
  - Granularity: Store + Week level (time-varying attributes)
  - Contains: Economic indicators, weather, promotions, holidays

#### **Duplicate column**
- **IsHoliday**: Column does not appear to be needed for referencial integrety, one option can be dropped.

In [8]:
# Drop duplicate IsHoliday column from features_df to avoid redundancy
# Keep IsHoliday in sales_df as it's the primary fact table
features_df_transform = features_df_transform.drop('IsHoliday', axis=1)

print("Dropped duplicate IsHoliday column from features dataset")

Dropped duplicate IsHoliday column from features dataset


In [9]:
# Check for orphaned keys between datasets
# Trim features table by last sales date
last_sales_date = sales_df_transform['Date'].max()
features_df_transform = features_df_transform[features_df_transform['Date'] <= last_sales_date]
print(f"Features table trimmed to end at {last_sales_date.strftime('%Y-%m-%d')}")

# Store comparisons
sales_stores = set(sales_df_transform['Store'].unique())
master_stores = set(stores_df_transform['Store'].unique())
orphaned_sales_stores = sales_stores - master_stores
orphaned_master_stores = master_stores - sales_stores

print(f"\nOrphaned stores in Sales (not in master): {len(orphaned_sales_stores)}")
print(f"Orphaned stores in Master (not in sales): {len(orphaned_master_stores)}")

# Store-date combinations
sales_keys = set(zip(sales_df_transform['Store'], sales_df_transform['Date']))
features_keys = set(zip(features_df_transform['Store'], features_df_transform['Date']))
orphaned_sales = sales_keys - features_keys
orphaned_features = features_keys - sales_keys

print(f"\nOrphaned sales combinations without features: {len(orphaned_sales):,}")
print(f"Orphaned features combinations without sales: {len(orphaned_features):,}")

Features table trimmed to end at 2012-10-26

Orphaned stores in Sales (not in master): 0
Orphaned stores in Master (not in sales): 0

Orphaned sales combinations without features: 0
Orphaned features combinations without sales: 0


### Joining the Tables

It appears that there aren't any orphaned keys, I can use LEFT JOIN or INNER JOIN with the same result

In [10]:
# Left join merge of definitions and feature definitions to the sales fact table

# Create complete dataset with all three tables joined
denormalised_df_transform = sales_df_transform.merge(stores_df_transform, on='Store', how='left') \
                           .merge(features_df_transform, on=['Store', 'Date'], how='left')

print(f"Denormalised dataset: {denormalised_df_transform.shape[0]:,} records, {denormalised_df_transform.shape[1]} columns")
print(f"Original sales records: {sales_df_transform.shape[0]:,} records, {sales_df_transform.shape[1]} columns")

# Verify join success - check for any null values in key columns
null_stores = denormalised_df_transform['Type'].isnull().sum()
null_features = denormalised_df_transform['Temperature'].isnull().sum()

print(f"\nJoin validation:")
print(f"Missing store data after join: {null_stores}")
print(f"Missing feature data after join: {null_features}")

# Display sample of joined data
print(f"\nHead of denormalised dataset:")
denormalised_df_transform.head()

Denormalised dataset: 421,570 records, 16 columns
Original sales records: 421,570 records, 5 columns

Join validation:
Missing store data after join: 0
Missing feature data after join: 0

Head of denormalised dataset:


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106


### Dataset Splitting Strategy

Based on the date range data quality issues identified above, we'll create **two analysis datasets** to maximize data utility:

#### **Dataset 1: General Analysis (Full Sales data Timeline)**
- **Purpose:** Sales trends, seasonality, store performance analysis
- **Timeline:** Complete sales data dataset (Feb 2010 - Oct 2012)
- **Features:** Store, Date, Temperature, Fuel_Price, CPI, Unemployment, IsHoliday, Weekly_Sales
- **Advantage:** Maximum data coverage for robust trend analysis

#### **Dataset 2: Promotion and Markdown Analysis (Nov 2011 onwards)**
- **Purpose:** Impact of markdowns and promotional strategies
- **Timeline:** From Nov 2011 when MarkDown data becomes available - Oct 2012
- **Features:** All features including MarkDown 1-5 columns
- **Advantage:** Complete feature set for promotional impact analysis

This approach allows me to:
- **Maximise data usage** - Use full timeline where appropriate
- **Maintain data quality** - Focus on complete records for markdown analysis
- **Enable comprehensive insights** - Compare pre/post promotional periods

---

#### Implementing the Split

**Naming Conventions:**
- General analysis datasets: Keep existing names (`denormalised_df`)
- Promotional analysis datasets: Use `promo_` prefix (`promo_denormalised_df`)

In [11]:
# Create promotional datasets (from first markdown date onwards) using the denormalised table
promo_denormalised_df_transform = denormalised_df_transform[denormalised_df_transform['Date'] >= first_markdown_date].copy()

# Create promotional datasets dictionary
promo_datasets = {
    'Promo_Complete': promo_denormalised_df_transform
}

# Generate data dictionary for promotional datasets
promo_data_dictionary = create_data_dictionary(promo_datasets)

print(f'Promotional dataset created from denormalised table starting {first_markdown_date.strftime("%Y-%m-%d")}:')

# Display promotional data dictionary filtered for rows with missing values
promo_data_dictionary[promo_data_dictionary['Missing Values'] > 0]

Promotional dataset created from denormalised table starting 2011-11-11:


,Dataset,Column,Data Type,Row Count,Missing Values,Missing %,Unique Values,Sample Values,Description
9,Promo_Complete,MarkDown1,float64,151432,751,0.50,2277,"10382.9, 6074.12, 410.31",Anonymized promotional markdown data (after No...
10,Promo_Complete,MarkDown2,float64,151432,40184,26.54,1499,"6115.67, 254.39, 98.0",Anonymized promotional markdown data (after No...
11,Promo_Complete,MarkDown3,float64,151432,14341,9.47,1662,"215.07, 51.98, 55805.51",Anonymized promotional markdown data (after No...
12,Promo_Complete,MarkDown4,float64,151432,16465,10.87,1944,"2406.62, 427.39, 8.0",Anonymized promotional markdown data (after No...


In [12]:
## Clean after or before split???
## Bais Analysis
## Imputing / Null Management Required
## Anomoly Analysis

In [13]:
# Create clean datasets from transformed data for analysis
sales_df = denormalised_df_transform.copy()
promo_sales_df = promo_denormalised_df_transform.copy()

# Clean up temporary transform dataframes - no longer needed
del sales_df_transform, features_df_transform, denormalised_df_transform, promo_denormalised_df_transform

print("\nTransform datasets cleaned up. Ready for analysis.")


Transform datasets cleaned up. Ready for analysis.


---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---